In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

The history saving thread hit an unexpected error (OperationalError('database is locked',)).History will not be written to the database.


In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

atmpro = saw
band = [4]
commitnumber = de5d43e
molecule = {'h2o': 'atmpro'}
tsfc = 257


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o h2o band3b
  atmpro = trp
  band = 3b
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 1e-24
  molecule = h2o
  ng_adju = [0]
  ng_refs = [6]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(600, 250)]
  tsfc = 294
  vmax = 720
  vmin = 620
  w_diffuse = [(1.66, 1.66, 1.66, 1.55, 1.5, 1.66)]
  wgt = [(0.8, 0.8, 0.8, 0.6, 0.6, 0.9)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312      1.0  0.000410                  NaN       NaN
0.000312      NaN       NaN                  1.0 -0.000097
0.000750      2.0  0.000465                  2.0 -0.000655
0.001052      3.0  0.000588                  3.0  0.000298
0.001476      4.0  0.000733                  4.0  0.000424
0.002070      5.0  0.000899                  5.0  0.000542
0.002904      6.0  0.001089                  6.0  0.000671
0.004074      7.0  0.001306                  7.0  0.000826
0.005714      8.0  0.001554                  8.0  0.001007
0.008015      9.0  0.001836                  9.0  0.001208
0.011243     10.0  0.002155                 10.0  0.001432
0.015771     11.0  0.002515                 11.0  0.001681
0.022122     12.0  0.002922                 12.0  0.001961
0.031031     13.0  0.003377                 13.0  0.002281
0.043528     14.0  0.003821                 14.0  0.002591
0.061057     15.0  0.004196                 15.0  0.002844
0.085645     16.0  0.004539                 16.0  0.003081
0.120136     17.0  0.004900                 17.0  0.003333
0.168516     18.0  0.005277                 18.0  0.003596
0.236378     19.0  0.005675                 19.0  0.003868
0.331549     20.0  0.006090                 20.0  0.004153
0.465100     21.0  0.006474                 21.0  0.004443
0.652400     22.0  0.006096                 22.0  0.004248
0.915100     23.0  0.004839                 23.0  0.003425
1.283650     24.0  0.003592                 24.0  0.002582
1.800600     25.0  0.002617                 25.0  0.001911
2.525700     26.0  0.001864                 26.0  0.001387
3.542800     27.0  0.001294                 27.0  0.000983
4.969550     28.0  0.000916                 28.0  0.000711
6.970850     29.0  0.000703                 29.0  0.000553
9.778100     30.0  0.000563                 30.0  0.000446
13.715850    31.0  0.000454                 31.0  0.000360
19.239350    32.0  0.000374                 32.0  0.000295
26.987250    33.0  0.000353                 33.0  0.000275
37.855300    34.0  0.000389                 34.0  0.000297
53.100050    35.0  0.000436                 35.0  0.000337
73.887500    36.0  0.000480                 36.0  0.000381
97.662500    37.0  0.000509                 37.0  0.000410
121.437500   38.0  0.000514                 38.0  0.000418
145.212500   39.0  0.000688                 39.0  0.000561
168.987500   40.0  0.000862                 40.0  0.000711
192.762500   41.0  0.001042                 41.0  0.000875
216.537500   42.0  0.001146                 42.0  0.000989
240.312500   43.0  0.001224                 43.0  0.001089
264.087500   44.0  0.001299                 44.0  0.001189
287.862500   45.0  0.001387                 45.0  0.001308
311.637500   46.0  0.001701                 46.0  0.001635
335.412500   47.0  0.002496                 47.0  0.002390
359.187500   48.0  0.003752                 48.0  0.003584
382.962500   49.0  0.006718                 49.0  0.006469
406.737500   50.0  0.011882                 50.0  0.011573
430.512500   51.0  0.018756                 51.0  0.018379
454.287500   52.0  0.027066                 52.0  0.026600
478.062500   53.0  0.036376                 53.0  0.035788
501.837500   54.0  0.046272                 54.0  0.045679
525.612500   55.0  0.056390                 55.0  0.055741
549.387500   56.0  0.066418                 56.0  0.065799
573.162500   57.0  0.076172                 57.0  0.075117
596.937500   58.0  0.085436                 58.0  0.083172
620.712500   59.0  0.094052                 59.0  0.090033
644.487500   60.0  0.101851                 60.0  0.096061
668.262500   61.0  0.108711                 61.0  0.101738
692.037500   62.0  0.114576                 62.0  0.107225
715.812500   63.0  0.119443                 63.0  0.112247
739.587500   64.0  0.123401                 64.0  0.116541
76

# Fluxes by Layer

CRD                              CLIRAD            \
                         flug          fldg      fnetg       flug      fldg   
pressure     level                                                            
0.000000e+00 1     -26.440621  0.000000e+00 -26.440621        NaN       NaN   
1.000000e-08 1            NaN           NaN        NaN -26.447472  0.000188   
6.244000e-04 2     -26.440621  5.335529e-08 -26.440621 -26.447472  0.000188   
8.759000e-04 3     -26.440621  7.618478e-08 -26.440621 -26.447472  0.000188   
1.228600e-03 4     -26.440621  1.123454e-07 -26.440621 -26.447472  0.000188   
1.723400e-03 5     -26.440621  1.696780e-07 -26.440621 -26.447472  0.000188   
2.417400e-03 6     -26.440621  2.604616e-07 -26.440621 -26.447472  0.000188   
3.390900e-03 7     -26.440621  4.040599e-07 -26.440621 -26.447472  0.000189   
4.756500e-03 8     -26.440621  6.309534e-07 -26.440621 -26.447472  0.000189   
6.672000e-03 9     -26.440621  9.890462e-07 -26.440620 -26.447472  0.000189   
9.358900e-03 10    -26.440621  1.553599e-06 -26.440620 -26.447472  0.000190   
1.312780e-02 11    -26.440621  2.442678e-06 -26.440619 -26.447472  0.000190   
1.841450e-02 12    -26.440621  3.841356e-06 -26.440617 -26.447472  0.000191   
2.583020e-02 13    -26.440621  6.039353e-06 -26.440615 -26.447472  0.000193   
3.623230e-02 14    -26.440620  9.487809e-06 -26.440611 -26.447472  0.000195   
5.082340e-02 15    -26.440619  1.481563e-05 -26.440604 -26.447471  0.000199   
7.129060e-02 16    -26.440617  2.286054e-05 -26.440594 -26.447470  0.000205   
1.000000e-01 17    -26.440613  3.486909e-05 -26.440578 -26.447468  0.000213   
1.402710e-01 18    -26.440608  5.275653e-05 -26.440555 -26.447464  0.000226   
1.967600e-01 19    -26.440599  7.933950e-05 -26.440520 -26.447459  0.000244   
2.759970e-01 20    -26.440585  1.187710e-04 -26.440466 -26.447450  0.000272   
3.871000e-01 21    -26.440563  1.770506e-04 -26.440386 -26.447436  0.000312   
5.431000e-01 22    -26.440529  2.624633e-04 -26.440267 -26.447414  0.000373   
7.617000e-01 23    -26.440484  3.749020e-04 -26.440109 -26.447386  0.000455   
1.068500e+00 24    -26.440436  5.031053e-04 -26.439933 -26.447358  0.000551   
1.498800e+00 25    -26.440392  6.416976e-04 -26.439750 -26.447334  0.000659   
2.102400e+00 26    -26.440353  7.902766e-04 -26.439563 -26.447318  0.000780   
2.949000e+00 27    -26.440324  9.484222e-04 -26.439376 -26.447313  0.000913   
4.136600e+00 28    -26.440310  1.116278e-03 -26.439194 -26.447323  0.001062   
5.802500e+00 29    -26.440314  1.301003e-03 -26.439013 -26.447353  0.001232   
8.139200e+00 30    -26.440338  1.519651e-03 -26.438818 -26.447406  0.001438   
1.141700e+01 31    -26.440390  1.790136e-03 -26.438600 -26.447491  0.001696   
1.601470e+01 32    -26.440484  2.131251e-03 -26.438352 -26.447621  0.002022   
2.246400e+01 33    -26.440640  2.572858e-03 -26.438067 -26.447815  0.002442   
3.151050e+01 34    -26.440878  3.189985e-03 -26.437688 -26.448096  0.003016   
4.420010e+01 35    -26.441221  4.117952e-03 -26.437103 -26.448487  0.003855   
6.200000e+01 36    -26.441709  5.525099e-03 -26.436184 -26.449047  0.005125   
8.577500e+01 37    -26.442373  7.539811e-03 -26.434833 -26.449816  0.006966   
1.095500e+02 38    -26.443055  9.654763e-03 -26.433401 -26.450606  0.008911   
1.333250e+02 39    -26.443779  1.182748e-02 -26.431952 -26.451428  0.010911   
1.571000e+02 40    -26.444845  1.483006e-02 -26.430015 -26.452606  0.013669   
1.808750e+02 41    -26.446331  1.874326e-02 -26.427588 -26.454208  0.017273   
2.046500e+02 42    -26.448351  2.369836e-02 -26.424652 -26.456342  0.021871   
2.284250e+02 43    -26.450867  2.944272e-02 -26.421425 -26.458972  0.027288   
2.522000e+02 44    -26.453932  3.595363e-02 -26.417979 -26.462149  0.033532   
2.759750e+02 45    -26.457681  4.335972e-02 -26.414321 -26.465989  0.040721   
2.997500e+02 46    -26.462259  5.184511e-02 -26.410414 -26.470609  0.049024   
3.235250e+02 47    -26.467725  6.210166e-02 -26.405623 -26.476064  0.059085 

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')